# Updated Predictions Notebook

## Prepare Notebook and Load Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import datetime as dt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
import copy

np.set_printoptions(precision=4)

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('use_inf_as_na', True)

In [2]:
# Import Data
races = pd.read_csv('./f1db_csv/races.csv')
results = pd.read_csv('./f1db_csv/results.csv')
qualifying = pd.read_csv('./f1db_csv/qualifying.csv')
driver_standings = pd.read_csv('./f1db_csv/driver_standings.csv')
constructor_standings = pd.read_csv('./f1db_csv/constructor_standings.csv')
constructors = pd.read_csv('./f1db_csv/constructors.csv')
drivers = pd.read_csv('./f1db_csv/drivers.csv')
driver_history = pd.read_csv('./f1db_csv/driver_history.csv')
lap_times = pd.read_csv('./f1db_csv/lap_times.csv')

### Get Weather Data from Wikipedia

In [3]:
# Scrape weather
weather = races.iloc[:,[0,1,2]]

# Scrape data for weather
info = []

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()
                        
                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima) 
                                
    except:
        info.append('not found')

In [4]:
# Create a df just for weather and clean it up
weather['weather'] = info
weather["weather"].iloc[1021] = "Warm, Sunny"
weather["weather"].iloc[1022] = "Warm, Sunny"
weather["weather"].iloc[1023] = "Warm, Sunny"
weather["weather"].iloc[1024] = "Rain, Cloudy"
weather["weather"].iloc[1025] = "Warm, Sunny"
weather["weather"].iloc[1026] = "Warm, Sunny"

weather_dict = {'weather_warm': ['soleggiato', 'clear', 'warm', 'hot', 'sunny', 'fine', 'mild', 'sereno'],
               'weather_cold': ['cold', 'fresh', 'chilly', 'cool'],
               'weather_dry': ['dry', 'asciutto'],
               'weather_wet': ['showers', 'wet', 'rain', 'pioggia', 'damp', 'thunderstorms', 'rainy'],
               'weather_cloudy': ['overcast', 'nuvoloso', 'clouds', 'cloudy', 'grey', 'coperto']}

weather_df = pd.DataFrame(columns = weather_dict.keys())

for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)

weather_info = pd.concat([weather, weather_df], axis = 1)
weather_info
weather_info.tail(15)

raceId  year  round        weather  weather_warm  weather_cold  \
1020    1033  2020      3  Partly cloudy             0             0   
1021    1034  2020      4    Warm, Sunny             1             0   
1022    1035  2020      5    Warm, Sunny             1             0   
1023    1036  2020      6    Warm, Sunny             1             0   
1024    1037  2020      7   Rain, Cloudy             0             0   
1025    1038  2020      8    Warm, Sunny             1             0   
1026    1039  2020      9    Warm, Sunny             1             0   
1027    1040  2020     10      not found             0             0   
1028    1041  2020     11      not found             0             0   
1029    1042  2020     12      not found             0             0   
1030    1043  2020     13      not found             0             0   
1031    1044  2020     14      not found             0             0   
1032    1045  2020     15      not found             0             0   
1033    1046  2020     16      not found             0             0   
1034    1047  2020     17      not found             0             0   

      weather_dry  weather_wet  weather_cloudy  
1020            0            0               1  
1021            0            0               0  
1022            0            0               0  
1023            0            0               0  
1024            0            0               1  
1025            0            0               0  
1026            0            0               0  
1027            0            0               0  
1028            0            0               0  
1029            0            0               0  
1030            0            0               0  
1031            0            0               0  
1032            0            0               0  
1033            0            0               0  
1034            0            0               0

In [5]:
# Write this to a csv to be used later
weather_info.to_csv("./f1db_csv/weather.csv")

In [7]:
weather = pd.read_csv("./f1db_csv/weather.csv")

## Create Tables for Predictions

In [57]:
# Merge together to create final df for predictions
df1 = pd.merge(races, weather[["raceId", "weather_warm", "weather_cold", "weather_dry", "weather_wet", "weather_cloudy"]], on=['raceId'], how = "left")
df2 = pd.merge(df1, results[["raceId", "driverId", "constructorId", "grid", "position", "points", "milliseconds", "laps", "fastestLapSpeed"]], on=['raceId'], how = "left")
df2 = df2.rename({'position' : 'finishingPosition', 'points' : 'pointsGained'}, axis = 1)
df3 = pd.merge(df2, driver_standings[["raceId", "driverId", "points", "position", "wins"]], on=['raceId', 'driverId'], how = "left")
df3 = df3.rename({'position' : 'driverStanding', 'points' : 'cumulativePoints', 'wins' : 'driver_wins'}, axis = 1)
df4 = pd.merge(df3, constructor_standings[["raceId", "constructorId", "points", "position", "wins"]], how='left', on=['raceId', 'constructorId']) #from 1958
df4 = df4.rename({'position' : 'constructorStanding', 'points' : 'constructorPoints', 'wins' : 'constructorWins', 'name' : 'raceName'}, axis = 1)
df5 = pd.merge(df4, constructors[["constructorId", "name"]], on = "constructorId", how = "left")
df5 = df5.rename({'name' : 'constructorName'}, axis = 1)

# Match drivers in driver history table to their driver IDs
drivers["driverName"] = drivers["forename"].str.cat(drivers["surname"],sep = " ")
df6 = pd.merge(driver_history, drivers[["driverName", "driverId"]], on = "driverName", how = "left")

final_df = pd.merge(df5, df6[["raceId", "driverId", "minQualifyingTime", "fastestLapTime"]], on=['raceId', 'driverId'], how = "left")
final_df = final_df.drop(columns = ["round", "circuitId", "time", "url", "constructorId"])

# Rearrange the columns so identifying information is at the beginning
key = lambda x: (x != 'raceId', x != 'date', x != 'year', x != 'raceName', x != 'driverId', x != 'finishingPosition')
final_df = final_df[sorted(final_df, key=key)]

In [58]:
# Get dummies for categorical variables
df_dum = pd.get_dummies(final_df, columns = ['constructorName'])
# Convert date to datetime
df_dum["date"] = pd.to_datetime(df_dum["date"])
df_dum = df_dum.sort_values("date", ascending = True)

In [63]:
# Merge lap times data
total_lap_times = pd.merge(lap_times, races[["raceId", "year"]], on = "raceId")
total_lap_times = total_lap_times[["raceId", "driverId", "milliseconds"]]
total_lap_times = total_lap_times.groupby(['raceId', 'driverId']).sum()
total_lap_times["seconds"] = total_lap_times.milliseconds / 1000

# Merge in the seconds for each race
df_dum_merged = pd.merge(df_dum, total_lap_times, on = ["raceId", "driverId"])
df_dum_merged = df_dum_merged.drop(columns = ["milliseconds_x", "milliseconds_y"])

# Look only at 2020 data right now
df_2020 = df_dum_merged[df_dum_merged.year == 2020]

# Drop teams that didn't exist in 2020
for col in df_2020.columns:
    if 'constructorName' in col and df_2020[col].sum() < 1:
        df_2020.drop(col, axis = 1, inplace = True)

df_2020 = df_2020.sort_values(["date", "driverId"], ascending = True)
df_2020

raceId       date  year             raceName  driverId  \
8973    1031 2020-07-05  2020  Austrian Grand Prix       1.0   
8960    1031 2020-07-05  2020  Austrian Grand Prix       8.0   
8974    1031 2020-07-05  2020  Austrian Grand Prix      20.0   
8966    1031 2020-07-05  2020  Austrian Grand Prix     154.0   
8975    1031 2020-07-05  2020  Austrian Grand Prix     815.0   
...      ...        ...   ...                  ...       ...   
9103    1038 2020-09-06  2020   Italian Grand Prix     844.0   
9112    1038 2020-09-06  2020   Italian Grand Prix     846.0   
9099    1038 2020-09-06  2020   Italian Grand Prix     847.0   
9100    1038 2020-09-06  2020   Italian Grand Prix     848.0   
9106    1038 2020-09-06  2020   Italian Grand Prix     849.0   

     finishingPosition  weather_warm  weather_cold  weather_dry  weather_wet  \
8973                 4             1             0            0            0   
8960                \N             1             0            0            0   
8974                10             1             0            0            0   
8966                \N             1             0            0            0   
8975                 6             1             0            0            0   
...                ...           ...           ...          ...          ...   
9103                \N             1             0            0            0   
9112                 4             1             0            0            0   
9099                14             1             0            0            0   
9100                15             1             0            0            0   
9106                11             1             0            0            0   

      weather_cloudy  grid  pointsGained  laps fastestLapSpeed  \
8973               0   5.0          12.0  71.0         229.572   
8960               0  19.0           0.0  53.0         225.185   
8974               0  11.0           1.0  71.0         226.524   
8966               0  15.0           0.0  49.0         221.347   
8975               0   6.0           8.0  71.0         227.579   
...              ...   ...           ...   ...             ...   
9103               0  13.0           0.0  23.0         242.424   
9112               0   6.0          12.0  53.0         247.587   
9099               0  19.0           0.0  53.0         247.033   
9100               0   9.0           0.0  53.0         245.564   
9106               0  20.0           0.0  53.0         245.353   

      cumulativePoints  driverStanding  driver_wins  constructorPoints  \
8973              12.0             4.0          0.0               37.0   
8960               0.0            14.0          0.0                2.0   
8974               1.0            10.0          0.0               19.0   
8966               0.0            16.0          0.0                0.0   
8975               8.0             6.0          0.0                8.0   
...                ...             ...          ...                ...   
9103              45.0             7.0          0.0               61.0   
9112              57.0             5.0          0.0               98.0   
9099               0.0            21.0          0.0                0.0   
9100              48.0             6.0          0.0              158.0   
9106               0.0            18.0          0.0                0.0   

      constructorStanding  constructorWins  minQualifyingTime  fastestLapTime  \
8973                  1.0              1.0             62.951          67.712   
8960                  7.0              0.0             65.224          69.031   
8974                  3.0              0.0             64.206          68.623   
8966                 10.0              0.0             64.691          70.228   
8975                  4.0              0.0             63.860          68.305   
...                   ...              ...                ...             ...   
9103                  6.0  

In [79]:
df_2020[df_2020.raceId == 1038]

raceId       date  year            raceName  driverId finishingPosition  \
9109    1038 2020-09-06  2020  Italian Grand Prix       1.0                 7   
9098    1038 2020-09-06  2020  Italian Grand Prix       8.0                13   
9104    1038 2020-09-06  2020  Italian Grand Prix      20.0                \N   
9097    1038 2020-09-06  2020  Italian Grand Prix     154.0                12   
9116    1038 2020-09-06  2020  Italian Grand Prix     815.0                10   
9110    1038 2020-09-06  2020  Italian Grand Prix     817.0                 6   
9111    1038 2020-09-06  2020  Italian Grand Prix     822.0                 5   
9101    1038 2020-09-06  2020  Italian Grand Prix     825.0                \N   
9107    1038 2020-09-06  2020  Italian Grand Prix     826.0                 9   
9102    1038 2020-09-06  2020  Italian Grand Prix     830.0                \N   
9114    1038 2020-09-06  2020  Italian Grand Prix     832.0                 2   
9108    1038 2020-09-06  2020  Italian Grand Prix     839.0                 8   
9113    1038 2020-09-06  2020  Italian Grand Prix     840.0                 3   
9105    1038 2020-09-06  2020  Italian Grand Prix     841.0                16   
9115    1038 2020-09-06  2020  Italian Grand Prix     842.0                 1   
9103    1038 2020-09-06  2020  Italian Grand Prix     844.0                \N   
9112    1038 2020-09-06  2020  Italian Grand Prix     846.0                 4   
9099    1038 2020-09-06  2020  Italian Grand Prix     847.0                14   
9100    1038 2020-09-06  2020  Italian Grand Prix     848.0                15   
9106    1038 2020-09-06  2020  Italian Grand Prix     849.0                11   

      weather_warm  weather_cold  weather_dry  weather_wet  weather_cloudy  \
9109             1             0            0            0               0   
9098             1             0            0            0               0   
9104             1             0            0            0               0   
9097             1             0            0            0               0   
9116             1             0            0            0               0   
9110             1             0            0            0               0   
9111             1             0            0            0               0   
9101             1             0            0            0               0   
9107             1             0            0            0               0   
9102             1             0            0            0               0   
9114             1             0            0            0               0   
9108             1             0            0            0               0   
9113             1             0            0            0               0   
9105             1             0            0            0               0   
9115             1             0            0            0               0   
9103             1             0            0            0               0   
9112             1             0            0            0               0   
9099             1             0            0            0               0   
9100             1             0            0            0               0   
9106             1             0            0            0               0   

      grid  pointsGained  laps fastestLapSpeed  cumulativePoints  \
9109   1.0           7.0  53.0         252.033             164.0   
9098  14.0           0.0  53.0         245.827               0.0   
9104  17.0           0.0   6.0         239.415              16.0   
9097  16.0           0.0  53.0         245.972               0.0   
9116   4.0           1.0  53.0         247.282              34.0   
9110   7.0           8.0  53.0         248.573              41.0   
9111   2.0          10.0  53.0         248.386             117.0   
9101  15.0           0.0  17.0         243.099               1.0   
9107  11.0           2.0  53.0         246.863       

In [102]:
# Reformat the data so that it's taking info from the previous race
# Separate the data from the current race and the previous race
current_df = df_2020[["raceId", "date", "year", "raceName", "driverId", "finishingPosition", "seconds", "weather_warm", "weather_cold", 
                      "weather_dry", "weather_wet", "weather_cloudy", "grid", "minQualifyingTime", 
                      "constructorName_Alfa Romeo", "constructorName_AlphaTauri", "constructorName_Ferrari", 
                      "constructorName_Haas F1 Team", "constructorName_McLaren", "constructorName_Mercedes", 
                      "constructorName_Racing Point", "constructorName_Red Bull", "constructorName_Renault", 
                      "constructorName_Williams"]]
current_df = current_df.rename({'finishingPosition' : 'currentFinishingPosition', 'grid' : 'qualiPosition'}, axis = 1)
past_df = df_2020[["raceId", "date", "year", "raceName", "driverId", "finishingPosition", "laps", "pointsGained", 
                   "fastestLapSpeed", "cumulativePoints", "driverStanding", "driver_wins", "constructorPoints", "constructorStanding",
                   "constructorWins", "fastestLapTime"]]
past_df = past_df.rename({'finishingPosition' : 'pastFinishingPosition', 'pointsGained' : 'pastPointsGained'}, axis = 1)

# Change raceId for the past races so that info from the previous race gets matched to the current race for preds
past_df["raceId"] = past_df["raceId"] + 1

# Merge the two together for predictions
full_df = pd.merge(current_df, past_df[["raceId", "driverId", "laps", "pastFinishingPosition", "pastPointsGained",
                                        "fastestLapSpeed", "cumulativePoints", "driverStanding", "driver_wins", 
                                        "constructorPoints", "constructorStanding", "constructorWins", "fastestLapTime"]],
                   on = ["raceId", "driverId"], how = "left")

# Create variable for the difference between qualifying position and finishing position
full_df = full_df.replace({r"\N": None, "NaN" : None})
# full_df["finishingPosition"] = pd.to_numeric(full_df["finishingPosition"])
# full_df["quali_final_delta"] = full_df["finishingPosition"] - full_df["grid"]
full_df[["driverId", "currentFinishingPosition", "pastFinishingPosition"]][full_df.raceId == 1038].sort_values("currentFinishingPosition")

driverId currentFinishingPosition pastFinishingPosition
152     842.0                        1                     8
142     815.0                       10                    10
157     849.0                       11                    16
141     154.0                       12                    15
139       8.0                       13                    12
155     847.0                       14                  None
156     848.0                       15                     6
151     841.0                       16                  None
148     832.0                        2                   NaN
150     840.0                        3                     9
154     846.0                        4                     7
144     822.0                        5                     2
143     817.0                        6                     4
138       1.0                        7                     1
149     839.0                        8                     5
146     826.0                        9                    11
140      20.0                     None                    13
145     825.0                     None                    17
147     830.0                     None                     3
153     844.0                     None                    14

In [103]:
full_df

raceId       date  year             raceName  driverId  \
0      1031 2020-07-05  2020  Austrian Grand Prix       1.0   
1      1031 2020-07-05  2020  Austrian Grand Prix       8.0   
2      1031 2020-07-05  2020  Austrian Grand Prix      20.0   
3      1031 2020-07-05  2020  Austrian Grand Prix     154.0   
4      1031 2020-07-05  2020  Austrian Grand Prix     815.0   
..      ...        ...   ...                  ...       ...   
153    1038 2020-09-06  2020   Italian Grand Prix     844.0   
154    1038 2020-09-06  2020   Italian Grand Prix     846.0   
155    1038 2020-09-06  2020   Italian Grand Prix     847.0   
156    1038 2020-09-06  2020   Italian Grand Prix     848.0   
157    1038 2020-09-06  2020   Italian Grand Prix     849.0   

    currentFinishingPosition   seconds  weather_warm  weather_cold  \
0                          4  5456.428             1             0   
1                       None  3980.856             1             0   
2                         10  5480.284             1             0   
3                       None  3695.277             1             0   
4                          6  5465.831             1             0   
..                       ...       ...           ...           ...   
153                     None  2127.678             1             0   
154                        4  6432.056             1             0   
155                       14  6462.649             1             0   
156                       15  6463.589             1             0   
157                       11  6458.932             1             0   

     weather_dry  weather_wet  weather_cloudy  qualiPosition  \
0              0            0               0            5.0   
1              0            0               0           19.0   
2              0            0               0           11.0   
3              0            0               0           15.0   
4              0            0               0            6.0   
..           ...          ...             ...            ...   
153            0            0               0           13.0   
154            0            0               0            6.0   
155            0            0               0           19.0   
156            0            0               0            9.0   
157            0            0               0           20.0   

     minQualifyingTime  constructorName_Alfa Romeo  \
0               62.951                           0   
1               65.224                           1   
2               64.206                           0   
3               64.691                           0   
4               63.860                           0   
..                 ...                         ...   
153             80.273                           0   
154             79.820                           0   
155             81.587                           0   
156             80.064                           0   
157             81.717                           0   

     constructorName_AlphaTauri  constructorName_Ferrari  \
0                             0                        0   
1                             0                        0   
2                             0                        1   
3                             0                        0   
4                             0                        0   
..                          ...                      ...   
153                           0                        1   
154                           0                        0   
155                           0                        0   
156                           0                        0   
157                           0                        0   

     constructorName_Haas F1 Team  constructorName_McLaren  \
0                               0                        0   
1                               0                        0   
2                               0                        0   
3                   

In [105]:
# Find the time for the fastest driver at every race
# Fastest Lap
fastest_time_df = full_df[full_df.pastFinishingPosition == "1"]
fastest_time_df = fastest_time_df[["raceId", "seconds", "laps"]]
fastest_time_df["avg_lap_time"] = fastest_time_df["seconds"] / fastest_time_df["laps"]
fastest_time_df

raceId   seconds  laps  avg_lap_time
26     1032  4984.402  71.0     70.202845
40     1033  5772.473  71.0     81.302437
60     1034  5281.283  70.0     75.446900
79     1035  4793.319  52.0     92.179212
108    1036  5529.456  52.0    106.335692
119    1037  5048.761  66.0     76.496379
138    1038  6443.301  44.0    146.438659

In [119]:
# Delete rows where there are nans
# full_df = full_df.replace({r"\N": None})
# full_df = full_df.dropna()

# Create ratios for all the variables in terms of seconds and then create a rolling average for them over the season
min_df = full_df.groupby("raceId", as_index = False)
min_lap_time = min_df.fastestLapTime.min()
temp1 = pd.merge(full_df, min_lap_time, on = "raceId")  
temp1["fastestLapRatio"] = temp1["fastestLapTime_x"] / temp1["fastestLapTime_y"]
temp1 = temp1.drop(columns = ["fastestLapTime_x", "fastestLapTime_y"])
min_quali_time = min_df.minQualifyingTime.min()
temp2 = pd.merge(temp1, min_quali_time, on = "raceId")
temp2["minQualiRatio"] = temp2["minQualifyingTime_x"] / temp2["minQualifyingTime_y"]
temp2 = temp2.drop(columns = ["minQualifyingTime_x", "minQualifyingTime_y"])
temp2["avg_lap_time"] = temp2["seconds"] / temp2["laps"]
temp_min = temp2.groupby("raceId", as_index = False)
min_avg_lap_time = temp_min.avg_lap_time.min()
temp_min1 = pd.merge(temp2, min_avg_lap_time, on = "raceId")
temp_min1["avgLapRatio"] = temp_min1["avg_lap_time_x"] / temp_min1["avg_lap_time_y"]
temp_min1 = temp_min1.drop(columns = ["avg_lap_time_x", "avg_lap_time_y"])
temp3 = temp_min1.groupby("driverId", as_index = True)
rolling_fastLapRatio = pd.DataFrame(temp3["fastestLapRatio"].rolling(window = 50, min_periods = 1).mean())
rolling_fastLapRatio = rolling_fastLapRatio.reset_index()
rolling_fastLapRatio = rolling_fastLapRatio.rename({"level_1" : "index", "fastestLapRatio" : "rolling_fastestLap"}, axis = 1)
rolling_minQualiRatio = pd.DataFrame(temp3["minQualiRatio"].rolling(window = 50, min_periods = 1).mean())
rolling_minQualiRatio = rolling_minQualiRatio.reset_index()
rolling_minQualiRatio = rolling_minQualiRatio.rename({"level_1" : "index", "minQualiRatio" : "rolling_minQualiRatio"}, axis = 1)
rolling_avgLapRatio = pd.DataFrame(temp3["avgLapRatio"].rolling(window = 50, min_periods = 1).mean())
rolling_avgLapRatio = rolling_avgLapRatio.reset_index()
rolling_avgLapRatio = rolling_avgLapRatio.rename({"level_1" : "index", "avgLapRatio" : "rolling_avgLapTime"}, axis = 1)
temp_min1 = temp_min1.reset_index()
temp4 = pd.merge(temp_min1, rolling_fastLapRatio, on = ["driverId", "index"])
temp5 = pd.merge(temp4, rolling_minQualiRatio, on = ["driverId", "index"])
temp6 = pd.merge(temp5, rolling_avgLapRatio, on = ["driverId", "index"])
temp6 = temp6.drop(columns = ["fastestLapRatio", "minQualiRatio", "avgLapRatio", "seconds", "laps"])

key = lambda x: (x != 'index', x != 'raceId', x != 'date', x != 'year', x != 'raceName', x != 'driverId', x != 'rolling_avgLapTime')
sorted_df = temp6[sorted(temp6, key=key)]
sorted_df["rolling_avgLapTime"] = sorted_df["rolling_avgLapTime"] * 100
sorted_df["rolling_fastestLap"] = sorted_df["rolling_fastestLap"] * 100
sorted_df["rolling_minQualiRatio"] = sorted_df["rolling_minQualiRatio"] * 100
sorted_df = sorted_df.drop(columns = "rolling_avgLapTime")
sorted_df[sorted_df.raceId == 1038]

index  raceId       date  year            raceName  driverId  \
138    138    1038 2020-09-06  2020  Italian Grand Prix       1.0   
139    139    1038 2020-09-06  2020  Italian Grand Prix       8.0   
140    140    1038 2020-09-06  2020  Italian Grand Prix      20.0   
141    141    1038 2020-09-06  2020  Italian Grand Prix     154.0   
142    142    1038 2020-09-06  2020  Italian Grand Prix     815.0   
143    143    1038 2020-09-06  2020  Italian Grand Prix     817.0   
144    144    1038 2020-09-06  2020  Italian Grand Prix     822.0   
145    145    1038 2020-09-06  2020  Italian Grand Prix     825.0   
146    146    1038 2020-09-06  2020  Italian Grand Prix     826.0   
147    147    1038 2020-09-06  2020  Italian Grand Prix     830.0   
148    148    1038 2020-09-06  2020  Italian Grand Prix     832.0   
149    149    1038 2020-09-06  2020  Italian Grand Prix     839.0   
150    150    1038 2020-09-06  2020  Italian Grand Prix     840.0   
151    151    1038 2020-09-06  2020  Italian Grand Prix     841.0   
152    152    1038 2020-09-06  2020  Italian Grand Prix     842.0   
153    153    1038 2020-09-06  2020  Italian Grand Prix     844.0   
154    154    1038 2020-09-06  2020  Italian Grand Prix     846.0   
155    155    1038 2020-09-06  2020  Italian Grand Prix     847.0   
156    156    1038 2020-09-06  2020  Italian Grand Prix     848.0   
157    157    1038 2020-09-06  2020  Italian Grand Prix     849.0   

    currentFinishingPosition  weather_warm  weather_cold  weather_dry  \
138                        7             1             0            0   
139                       13             1             0            0   
140                     None             1             0            0   
141                       12             1             0            0   
142                       10             1             0            0   
143                        6             1             0            0   
144                        5             1             0            0   
145                     None             1             0            0   
146                        9             1             0            0   
147                     None             1             0            0   
148                        2             1             0            0   
149                        8             1             0            0   
150                        3             1             0            0   
151                       16             1             0            0   
152                        1             1             0            0   
153                     None             1             0            0   
154                        4             1             0            0   
155                       14             1             0            0   
156                       15             1             0            0   
157                       11             1             0            0   

     weather_wet  weather_cloudy  qualiPosition  constructorName_Alfa Romeo  \
138            0               0            1.0                           0   
139            0               0           14.0                           1   
140            0               0           17.0                           0   
141            0               0           16.0                           0   
142            0               0            4.0                           0   
143            0               0            7.0                           0   
144            0               0            2.0                           0   
145            0               0           15.0                           0   
146            0               0           11.0                           0   
147            0               0            5.0                           0   
148            0               0            3.0                           0   
149            0               0           12.0                          

## Build the Model

In [123]:
# Split x and y in training data
sorted_df["fastestLapSpeed"] = pd.to_numeric(sorted_df["fastestLapSpeed"])
sorted_df["currentFinishingPosition"] = pd.to_numeric(sorted_df["currentFinishingPosition"])
sorted_df["pastFinishingPosition"] = pd.to_numeric(sorted_df["pastFinishingPosition"])

# Remove any NAs
sorted_df = sorted_df.replace({r"\N": None})
sorted_df = sorted_df.dropna()

x = sorted_df.iloc[:, 7:]
y = sorted_df.iloc[:, 6]

In [124]:
xgb_model = xgb.XGBRegressor(n_estimators = 300, max_depth = 9)
xgb_model.fit(x, y)
xgb_model.score(x, y)

0.999999976622024

In [126]:
# Doing the same for the most recent race, pulled in from Raw Predictive Modeling ipynb
# Merge lap times data
total_lap_times = pd.merge(lap_times, races[["raceId", "year"]], on = "raceId")
total_lap_times = total_lap_times[["raceId", "driverId", "milliseconds"]]
total_lap_times = total_lap_times.groupby(['raceId', 'driverId']).sum()
total_lap_times["seconds"] = total_lap_times.milliseconds / 1000
total_lap_info = pd.merge(total_lap_times, df_dum[["raceId", "driverId", "laps"]], on = ["raceId", "driverId"])
total_lap_info["avg_lap_time"] = total_lap_info.seconds / total_lap_info.laps
total_lap_info

test_df = pd.read_csv("./predictions/tu_2020_raw_table").iloc[:, 1:]
past_race_lap_info = total_lap_info[total_lap_info.raceId == 1038]
past_race_lap_info["raceId"] = past_race_lap_info["raceId"] + 1
past_race_lap_info
test_merged_df = pd.merge(test_df, past_race_lap_info[["raceId", "driverId", "seconds"]], on = ["raceId", "driverId"])
test_merged_df

raceId  driverId  weather_warm  weather_cold  weather_dry  weather_wet  \
0     1039         1             1             0            0            0   
1     1039       822             1             0            0            0   
2     1039       830             1             0            0            0   
3     1039       848             1             0            0            0   
4     1039       844             1             0            0            0   
5     1039        20             1             0            0            0   
6     1039       815             1             0            0            0   
7     1039       840             1             0            0            0   
8     1039       817             1             0            0            0   
9     1039       839             1             0            0            0   
10    1039       832             1             0            0            0   
11    1039       846             1             0            0            0   
12    1039       826             1             0            0            0   
13    1039       842             1             0            0            0   
14    1039         8             1             0            0            0   
15    1039       841             1             0            0            0   
16    1039       154             1             0            0            0   
17    1039       825             1             0            0            0   
18    1039       847             1             0            0            0   
19    1039       849             1             0            0            0   

    weather_cloudy  grid  minQualifyingTime  constructorName_Alfa Romeo  \
0                0     1             75.144                           0   
1                0     2             75.203                           0   
2                0     3             75.471                           0   
3                0     4             75.914                           0   
4                0     5             76.270                           0   
5                0    14             76.858                           0   
6                0     6             76.311                           0   
7                0     7             76.271                           0   
8                0     8             76.243                           0   
9                0    10             76.297                           0   
10               0     9             76.522                           0   
11               0    11             76.640                           0   
12               0    12             76.854                           0   
13               0    16             77.125                           0   
14               0    13             76.854                           1   
15               0    17             77.220                           1   
16               0    15             77.069                           0   
17               0    20             77.348                           0   
18               0    18             77.232                           0   
19               0    19             77.320                           0   

    constructorName_AlphaTauri  constructorName_Ferrari  \
0                            0                        0   
1                            0                        0   
2                            0                        0   
3                            0                        0   
4                            0                        1   
5                            0                        1   
6                            0                        0   
7                            0                        0   
8                            0                        0   
9                            0                        0   
10                           0                        0   
11                           0                        0   
12                    

In [143]:
current_min_df = test_merged_df.groupby("raceId", as_index = False)
current_min_lap_time = current_min_df.fastestLapTime.min()
current_temp1 = pd.merge(test_merged_df, current_min_lap_time, on = "raceId")  
current_temp1["fastestLapRatio"] = current_temp1["fastestLapTime_x"] / current_temp1["fastestLapTime_y"]
current_temp1 = current_temp1.drop(columns = ["fastestLapTime_x", "fastestLapTime_y"])
current_min_quali_time = current_min_df.minQualifyingTime.min()
current_temp2 = pd.merge(current_temp1, current_min_quali_time, on = "raceId")
current_temp2["minQualiRatio"] = current_temp2["minQualifyingTime_x"] / current_temp2["minQualifyingTime_y"]
current_temp2 = current_temp2.drop(columns = ["minQualifyingTime_x", "minQualifyingTime_y"])
current_temp2["avg_lap_time"] = current_temp2["seconds"] / current_temp2["laps"]
current_temp_min = current_temp2.groupby("raceId", as_index = False)
current_min_avg_lap_time = current_temp_min.avg_lap_time.min()
current_temp_min1 = pd.merge(current_temp2, current_min_avg_lap_time, on = "raceId")
current_temp_min1["avgLapRatio"] = current_temp_min1["avg_lap_time_x"] / current_temp_min1["avg_lap_time_y"]
current_temp_min1 = current_temp_min1.drop(columns = ["avg_lap_time_x", "avg_lap_time_y"])
current_temp3 = current_temp_min1.groupby("driverId", as_index = True)
current_rolling_fastLapRatio = pd.DataFrame(current_temp3["fastestLapRatio"].rolling(window = 50, min_periods = 1).mean())
current_rolling_fastLapRatio = current_rolling_fastLapRatio.reset_index()
current_rolling_fastLapRatio = current_rolling_fastLapRatio.rename({"level_1" : "index", "fastestLapRatio" : "rolling_fastestLap"}, axis = 1)
current_rolling_minQualiRatio = pd.DataFrame(current_temp3["minQualiRatio"].rolling(window = 50, min_periods = 1).mean())
current_rolling_minQualiRatio = current_rolling_minQualiRatio.reset_index()
current_rolling_minQualiRatio = current_rolling_minQualiRatio.rename({"level_1" : "index", "minQualiRatio" : "rolling_minQualiRatio"}, axis = 1)
current_rolling_avgLapRatio = pd.DataFrame(current_temp3["avgLapRatio"].rolling(window = 50, min_periods = 1).mean())
current_rolling_avgLapRatio = current_rolling_avgLapRatio.reset_index()
current_rolling_avgLapRatio = current_rolling_avgLapRatio.rename({"level_1" : "index", "avgLapRatio" : "rolling_avgLapTime"}, axis = 1)
current_temp_min1 = current_temp_min1.reset_index()
current_temp4 = pd.merge(current_temp_min1, current_rolling_fastLapRatio, on = ["driverId", "index"])
current_temp5 = pd.merge(current_temp4, current_rolling_minQualiRatio, on = ["driverId", "index"])
current_temp6 = pd.merge(current_temp5, current_rolling_avgLapRatio, on = ["driverId", "index"])
current_temp6 = current_temp6.drop(columns = ["fastestLapRatio", "minQualiRatio", "avgLapRatio", "seconds", "laps"])

key = lambda x: (x != 'index', x != 'raceId', x != 'date', x != 'year', x != 'raceName', x != 'driverId', x != 'rolling_avgLapTime')
current_sorted_df = current_temp6[sorted(current_temp6, key=key)]
current_sorted_df["rolling_avgLapTime"] = current_sorted_df["rolling_avgLapTime"] * 100
current_sorted_df["rolling_fastestLap"] = current_sorted_df["rolling_fastestLap"] * 100
current_sorted_df["rolling_minQualiRatio"] = current_sorted_df["rolling_minQualiRatio"] * 100
# current_sorted_df = current_sorted_df.dropna()
current_sorted_df = current_sorted_df.drop(columns = "rolling_avgLapTime")
current_sorted_df = current_sorted_df.rename({'finishingPosition' : 'pastFinishingPosition', 'grid' : 'qualiPosition', 'pointsGained' : 'pastPointsGained'}, axis = 1)
current_sorted_df

index  raceId  driverId  weather_warm  weather_cold  weather_dry  \
0       0    1039         1             1             0            0   
1       1    1039       822             1             0            0   
2       2    1039       830             1             0            0   
3       3    1039       848             1             0            0   
4       4    1039       844             1             0            0   
5       5    1039        20             1             0            0   
6       6    1039       815             1             0            0   
7       7    1039       840             1             0            0   
8       8    1039       817             1             0            0   
9       9    1039       839             1             0            0   
10     10    1039       832             1             0            0   
11     11    1039       846             1             0            0   
12     12    1039       826             1             0            0   
13     13    1039       842             1             0            0   
14     14    1039         8             1             0            0   
15     15    1039       841             1             0            0   
16     16    1039       154             1             0            0   
17     17    1039       825             1             0            0   
18     18    1039       847             1             0            0   
19     19    1039       849             1             0            0   

    weather_wet  weather_cloudy  qualiPosition  constructorName_Alfa Romeo  \
0             0               0              1                           0   
1             0               0              2                           0   
2             0               0              3                           0   
3             0               0              4                           0   
4             0               0              5                           0   
5             0               0             14                           0   
6             0               0              6                           0   
7             0               0              7                           0   
8             0               0              8                           0   
9             0               0             10                           0   
10            0               0              9                           0   
11            0               0             11                           0   
12            0               0             12                           0   
13            0               0             16                           0   
14            0               0             13                           1   
15            0               0             17                           1   
16            0               0             15                           0   
17            0               0             20                           0   
18            0               0             18                           0   
19            0               0             19                           0   

    constructorName_AlphaTauri  constructorName_Ferrari  \
0                            0                        0   
1                            0                        0   
2                            0                        0   
3                            0                        0   
4                            0                        1   
5                            0                        1   
6                            0                        0   
7                            0                        0   
8                            0                        0   
9                            0                        0   
10                           0                        0   
11                           0                        0   
12                           1                        0   
13                        

In [144]:
x_test = current_sorted_df.iloc[:, 3:]

In [145]:
x_test

weather_warm  weather_cold  weather_dry  weather_wet  weather_cloudy  \
0              1             0            0            0               0   
1              1             0            0            0               0   
2              1             0            0            0               0   
3              1             0            0            0               0   
4              1             0            0            0               0   
5              1             0            0            0               0   
6              1             0            0            0               0   
7              1             0            0            0               0   
8              1             0            0            0               0   
9              1             0            0            0               0   
10             1             0            0            0               0   
11             1             0            0            0               0   
12             1             0            0            0               0   
13             1             0            0            0               0   
14             1             0            0            0               0   
15             1             0            0            0               0   
16             1             0            0            0               0   
17             1             0            0            0               0   
18             1             0            0            0               0   
19             1             0            0            0               0   

    qualiPosition  constructorName_Alfa Romeo  constructorName_AlphaTauri  \
0               1                           0                           0   
1               2                           0                           0   
2               3                           0                           0   
3               4                           0                           0   
4               5                           0                           0   
5              14                           0                           0   
6               6                           0                           0   
7               7                           0                           0   
8               8                           0                           0   
9              10                           0                           0   
10              9                           0                           0   
11             11                           0                           0   
12             12                           0                           1   
13             16                           0                           1   
14             13                           1                           0   
15             17                           1                           0   
16             15                           0                           0   
17             20                           0                           0   
18             18                           0                           0   
19             19                           0                           0   

    constructorName_Ferrari  constructorName_Haas F1 Team  \
0                         0                             0   
1                         0                             0   
2                         0                             0   
3                         0                             0   
4                         1                             0   
5                         1                             0   
6                         0                             0   
7                         0                             0   
8                         0                             0   
9                         0                             0   
10                        0                             0   
11                        0                             0

In [146]:
xgb_preds = xgb_model.predict(x_test)
xgb_preds_df = pd.DataFrame({'Predictions' : xgb_preds})
merged_xgb_df = pd.merge(xgb_preds_df, current_sorted_df[["raceId", "driverId"]], left_index = True, right_index = True)
full_xgb_df = pd.merge(merged_xgb_df, drivers[["driverName", "driverId"]], on = "driverId")
full_xgb_df = full_xgb_df.sort_values("Predictions")
full_xgb_df = full_xgb_df.reset_index(drop = True)
full_xgb_df["Predicted Position"] = full_xgb_df.index + 1
full_xgb_df.sort_values("Predictions")

Predictions  raceId  driverId          driverName  Predicted Position
0      1.652876    1039       822     Valtteri Bottas                   1
1      1.765163    1039       848     Alexander Albon                   2
2      1.803732    1039       830      Max Verstappen                   3
3      2.893155    1039         1      Lewis Hamilton                   4
4      4.898154    1039       839        Esteban Ocon                   5
5      5.178482    1039       826        Daniil Kvyat                   6
6      5.934350    1039       815        Sergio Pérez                   7
7      6.853724    1039       844     Charles Leclerc                   8
8      6.854187    1039       832        Carlos Sainz                   9
9      6.912608    1039       846        Lando Norris                  10
10     7.300628    1039       817    Daniel Ricciardo                  11
11     7.336318    1039       840        Lance Stroll                  12
12     7.533558    1039       842        Pierre Gasly                  13
13     8.320702    1039        20    Sebastian Vettel                  14
14     9.342052    1039         8      Kimi Räikkönen                  15
15    11.759418    1039       154     Romain Grosjean                  16
16    15.181354    1039       841  Antonio Giovinazzi                  17
17    15.224412    1039       849     Nicholas Latifi                  18
18    15.433035    1039       825     Kevin Magnussen                  19
19    16.237057    1039       847      George Russell                  20